

> authored by Ena Macahiya



# Random Forest Overview
- supervised learning, handles regression and classification
- ensemble learning method - combine predictions from other models. each smaller model in random forest ensemble is a **decision tree**
- how it works: multiple trees are created using random subsets of data and features. each DT is an expert providing its opinion on how to classify the data. predictions are made by calculating prediction for each DT and returning most popular or average result

In [ ]:
import pandas as pd
import numpy as np

# model imports
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import randint

In [8]:
# import preprocessed data
X_train = pd.read_csv("D:\\AI4ALL-Group-8C\\data\\processed\\X_train.csv")
X_test = pd.read_csv("D:\\AI4ALL-Group-8C\\data\\processed\\X_test.csv")
X_val = pd.read_csv("D:\\AI4ALL-Group-8C\\data\\processed\\X_val.csv")
# req 1D series for fit
y_train = pd.read_csv("D:\\AI4ALL-Group-8C\\data\\processed\\y_train.csv").iloc[:, 0]
y_test = pd.read_csv("D:\\AI4ALL-Group-8C\\data\\processed\\y_test.csv").iloc[:, 0]
y_val = pd.read_csv("D:\\AI4ALL-Group-8C\\data\\processed\\y_val.csv").iloc[:, 0]

In [17]:
# fit model w hyperparam tuning
param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

rf = RandomForestRegressor(random_state=42) # params can use n_estimators and random_state to have same result

rand_search = RandomizedSearchCV(rf, # model
                                 param_distributions = param_dist, # dict of hyp ranges to sample from
                                 n_iter=5, # num of random combos
                                 cv=5, # num of folds in cross-validation
                                 random_state=42, # seed
                                 n_jobs=4, # 4 CPU cores
                                 verbose=2) # shows progress

rand_search.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=5, n_jobs=4,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000015ACCBACC20>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000015ACC2CDA30>},
                   random_state=42, verbose=2)

In [18]:
# best model from random hyp test
best_rf = rand_search.best_estimator_
print('Best hyperparameters:',  rand_search.best_params_)

Best hyperparameters: {'max_depth': 8, 'n_estimators': 238}


In [19]:
# val predictions and acc
y_pred = best_rf.predict(X_val)
print("MSE:", mean_squared_error(y_val, y_pred))
print("R²:", r2_score(y_val, y_pred))

MSE: 2.807485513926882
R²: 0.9996988903405571


In [27]:
# test predictions and acc
y_pred = best_rf.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))

MSE: 2.571327938230775
R²: 0.9997318919803415


# Rundown of Accuracy Calculations
MSE = the closer to zero, the better. lower = better fit of the model to the data. measures the avg of the squares of the errors or deviations. for each data point find this: (actual value - predicted value)^2 and add it all together. then divide by the number of data points. </br>
R^2 = the higher, the better. if its 1, the model explains all the variability of the dependent variable around its mean. represents the proportion of the variance in the dependent variable that can be explained by the independent variables in a regression model. (how well does the independent var explain the variability of the dependent varaible). the formula is a little more complicated but in essence you divide the sum of squares of residuals over the total sum of squares, and subtract it from 1.

# Saving the Model

In [ ]:
import joblib
import os 

model_dir = 'D:\\AI4ALL-Group-8C\\models'
model_filename = 'best_random_forest_regressor.joblib'
model_path = os.path.join(model_dir, model_filename)

os.makedirs(model_dir, exist_ok=True)

# save the model
joblib.dump(best_rf, model_path)
print(f"model saved to {model_path}")

model saved to D:\AI4ALL-Group-8C\models\best_random_forest_regressor.joblib


In [ ]:
# model loading (for use in a new session or after training is done)
# to load the model, uncomment if running the full notebook without re-training.

# loaded_model = joblib.load(model_path)
# print("model loaded successfully!")

# # example: Test predictions and accuracy using the loaded model
# y_pred_loaded_test = loaded_model.predict(X_test)
# print("\nMetrics from Loaded Model (Test Set)")
# print("MSE (Test):", mean_squared_error(y_test, y_pred_loaded_test))
# print("R² (Test):", r2_score(y_test, y_pred_loaded_test))